In [259]:
#SERVER_URL=https://dataverse.ucla.edu
#PERSISTENT_IDENTIFIER=doi:10.25346/S6/T4LHZF
#SIZE=10000000
#
# token set programmatically

In [260]:
from datetime import datetime
import json   # used to pull in dataverse token
import requests  # http://docs.python-requests.org/en/master/

In [261]:
# --------------------------------------------------
# Update the 4 params below to run this code
# --------------------------------------------------
dataverse_server = 'https://dataverse.ucla.edu' # no trailing slash
dataset_id = 1  # database id of the dataset
persistentId = 'doi:10.25346/S6/T4LHZF' # doi or hdl of the dataset   10.25346/S6/T4LHZF,
size=10000000

In [262]:
# api in separate file
with open("config.json") as config:
    cfg = json.load(config)
# load either dataverse_production or dataverse_test
api_key = cfg['dataverse_production']['api_token']

In [263]:
# --------------------------------------------------
# Prepare "file"
# --------------------------------------------------
file_content = 'content: %s' % datetime.now()
files = {'file': ('sample_file.txt', file_content)}

# --------------------------------------------------
# Using a "jsonData" parameter, add optional description + file tags
# --------------------------------------------------
params = dict(description="LARIAC4", categories=["LARIAC4", "GIS","LAS file", "Lidar"], contentType="LAS file Lidar", restrict=True)

params_as_json_string = json.dumps(params)

payload = dict(jsonData=params_as_json_string)



In [264]:
# --------------------------------------------------
# Add file using the Dataset's persistentId (e.g. doi, hdl, etc)
# --------------------------------------------------
#url_persistent_id = '%s/api/datasets/:persistentId/add?persistentId=%s' % (dataverse_server, persistentId, api_key)

# -------------------
# Update the file content to avoid a duplicate file error
# -------------------
file_content = 'content2: %s' % datetime.now()
files = {'file': ('L4_6292_1842a.las', file_content)}


# -------------------
# Make the request
# -------------------

#r = requests.post(url_persistent_id, data=payload, files=files)

# -------------------
# Print the response
# -------------------

#print(r.jsonr.status_code())
#print() r.status_code

# --------------------------------------------------
# Add file using the Dataset's id
# --------------------------------------------------
#url_dataset_id = '%s/api/datasets/%s/add?key=%s' % (dataverse_server, dataset_id, api_key)
#r = requests.post(url_persistent_id, data=payload, files=files)

In [265]:
#  Ok, now do this with the directupload api
# https://guides.dataverse.org/en/5.4/developers/s3-direct-upload-api.html
#https://guides.dataverse.org/en/5.4/developers/s3-direct-upload-api.html

In [270]:
# Part 1: Requesting Direct Upload of a DataFile
# curl -H 'X-Dataverse-key:$API_TOKEN' 
#      "$SERVER_URL/api/datasets/:persistentId/uploadurls?persistentId=$PERSISTENT_IDENTIFIER&size=$SIZE"
# get pieces and make request
#SERVER_URL=https://dataverse.ucla.edu
#PERSISTENT_IDENTIFIER=doi:10.25346/S6/T4LHZF
#SIZE=10000000
dataverse_server = 'https://dataverse.ucla.edu' # no trailing slash
persistentId = 'doi:10.25346/S6/T4LHZF' # doi or hdl of the dataset
size=10000000

# --------------------------------------------------
# Add file using the Dataset's persistentId (e.g. doi, hdl, etc)
# --------------------------------------------------
#    url_dataset_id = '%s/api/datasets/%s/add?key=%s' % (dataverse_server, dataset_id, api_key)
#  $SERVER_URL/api/datasets/:persistentId/uploadurls?persistentId=$PERSISTENT_IDENTIFIER&size=$SIZE"
url_persistent_id = '%s/api/datasets/:persistentId/uploadurls?persistentId=%s&size=%s' % (dataverse_server, persistentId, str(size))

#url_persistent_id = '%s/api/datasets/:persistentId/uploadurls?key=%s?persistentId=%s&size=%s' % (dataverse_server, api_key, persistentId, str(size))


headers = {'X-Dataverse-key': api_key}
# make this into a request:
# 'X-Dataverse-key:$API_TOKEN' 
#      "/api/datasets/:persistentId/uploadurls?persistentId=$persistentId&size=$size"
#  
r = requests.put(url_persistent_id,
    headers={
        "X-Dataverse-key": "$API_TOKEN"
    },
    cookies={},
    auth=(),
)

In [271]:
print(r.status_code)

405


In [272]:
import pyDataverse
from pyDataverse.api import NativeApi
api = NativeApi(dataverse_server, api_key)
import subprocess as sp
from requests import ConnectionError, Response, delete, get, post, put
resp = api.get_info_version()
resp.json()

{'status': 'OK', 'data': {'version': '5.3', 'build': '286-fcb5ce7'}}

In [273]:
resp = requests.put(url_persistent_id, data=None, params=None, auth=(), files=None)
resp.json()

{'status': 'ERROR',
 'code': 405,
 'message': 'API endpoint does not support this method. Consult our API guide at http://guides.dataverse.org.',
 'requestUrl': 'https://dataverse.ucla.edu/api/v1/datasets/:persistentId/uploadurls?persistentId=doi:10.25346/S6/T4LHZF&size=10000000',
 'requestMethod': 'PUT'}

In [228]:
# Part 2: Adding the Uploaded file to the Dataset
# Once the file exists in the s3 bucket, a final API call is needed to add it to the Dataset. 
# This call is the same call used to upload a file to a Dataverse installation but, rather than sending the file bytes, 
# additional metadata is added using the “jsonData” parameter. jsonData normally includes information such as 
# a file description, tags, provenance, whether the file is restricted, etc. For direct uploads, 
# the jsonData object must also include values for:
# “storageIdentifier” - String, as specified in prior calls
# “fileName” - String
# “mimeType” - String
# “fileSize” - number of bytes
# fixity/checksum: either:
# “md5Hash” - String with MD5 hash value, or
# “checksum” - Json Object with “@type” field specifying the algorithm used and “@value” field with the 
#   value from that algorithm, both Strings
# The allowed checksum algorithms are defined by the edu.harvard.iq.dataverse.DataFile.CheckSumType class and 
#  currently include MD5, SHA-1, SHA-256, and SHA-512
# curl -X POST -H 'X-Dataverse-key: $API_TOKEN' 
#  "$SERVER_URL/api/datasets/:persistentId/add?persistentId=#PERSISTENT_IDENTIFIER" -F 'jsonData=$JSON_DATA'

In [250]:
import uncurl
print(uncurl.parse("curl -H 'X-Dataverse-key:$API_TOKEN' '$SERVER_URL/api/datasets/:persistentId/uploadurls?persistentId=$PERSISTENT_IDENTIFIER&size=$SIZE'"))

requests.get("$SERVER_URL/api/datasets/:persistentId/uploadurls?persistentId=$PERSISTENT_IDENTIFIER&size=$SIZE",
    headers={
        "X-Dataverse-key": "$API_TOKEN"
    },
    cookies={},
    auth=(),
)
